In [6]:
import geopandas as gpd
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import h5py
from numpy import dtype
from pandas import RangeIndex

# This notebook will be used to:
- #### Aggregate PSRC's TAZ to the ZIPCODEs
- #### Get the OD matrices to the ZIP-code level

# Aggregating PSRC's TAZs

First I'm gonna assign any TAZ that has its centroid inside the Zip code to that zipcode

After that, as there are some that have funny borders with the water, I'm gonna do the ones that intersect the Zipcode be assigned to that zipcode

In [46]:
#Importing the TAZ
TAZ = gpd.read_file('data/PSRC/soundcast_042125.gdb', layer = 'TAZ_2010')
# TAZ = TAZ.drop(columns = ['park', 'acres', 'GlobalID', 'Shape_Length', 'Shape_Area'])
TAZ_original = TAZ.copy()

#Importing the zipcodes in the 2010 format
ZIP_code = gpd.read_file('data/ZIP Codes/zip10_full.gpkg')
ZIP_code = ZIP_code[['GEOID10', 'geometry']]
ZIP_code_original = ZIP_code.copy()

In [8]:
TAZ

,taz,park,acres,GlobalID,Shape_Length,Shape_Area,geometry
0,1.0,0.0,1758.710924,{F3202174-57CD-4E95-AECB-18D9AB20377F},46720.500071,7.660914e+07,"MULTIPOLYGON (((1264477.853 269042.424, 126446..."
1,2.0,0.0,120.755587,{D5383690-0E2A-43B7-81DC-A17CFA9645B0},10591.087634,5.260092e+06,"MULTIPOLYGON (((1265788.093 268180.249, 126577..."
2,3.0,0.0,124.332936,{4A65D6B2-0DFD-447F-B74C-8A2233E81DBF},9555.337706,5.415921e+06,"MULTIPOLYGON (((1265848.689 271492.146, 126617..."
3,4.0,0.0,112.086215,{21D4918A-BD56-4A8B-A59F-772BD6734ED0},9618.098623,4.882456e+06,"MULTIPOLYGON (((1267095.215 269151.901, 126728..."
4,5.0,0.0,57.157023,{40947E9A-D688-4C0D-A90B-70BCAA660DE0},7807.650622,2.489750e+06,"MULTIPOLYGON (((1270732.726 269633.63, 1271063..."
...,...,...,...,...,...,...,...
3695,3696.0,0.0,985.847070,{2363DDFD-B98C-4A69-A871-C38D6885885C},30514.017509,4.294333e+07,"MULTIPOLYGON (((1213639.678 239372.617, 121371..."
3696,3697.0,0.0,2609.642770,{564442F1-C923-4F7D-8D18-44DDE3C9978F},43581.705129,1.136756e+08,"MULTIPOLYGON (((1212031.956 234139.577, 121203..."
3697,3698.0,0.0,641.622735,{A668A34C-6202-41A1-B839-736188CE6670},31374.018528,2.794898e+07,"MULTIPOLYGON (((1219221.496 231349.083, 121923..."
3698,3699.0,0.0,3095.122859,{273D851F-C04B-4029-8469-73AD6F8497C4},65860.215701,1.348230e+08,"MULTIPOLYGON (((1223293.588 230473.96, 1224169..."


In [9]:
#Working on this projection to make all the centroids and intersections
epsg_work = 26910

In [10]:
#Making the centroids
TAZ['geometry'] = TAZ_original['geometry'].to_crs(epsg_work).centroid
TAZ


,taz,park,acres,GlobalID,Shape_Length,Shape_Area,geometry
0,1.0,0.0,1758.710924,{F3202174-57CD-4E95-AECB-18D9AB20377F},46720.500071,7.660914e+07,POINT (545537.557 5286129.922)
1,2.0,0.0,120.755587,{D5383690-0E2A-43B7-81DC-A17CFA9645B0},10591.087634,5.260092e+06,POINT (548122.031 5286344.538)
2,3.0,0.0,124.332936,{4A65D6B2-0DFD-447F-B74C-8A2233E81DBF},9555.337706,5.415921e+06,POINT (548718.019 5286633.57)
3,4.0,0.0,112.086215,{21D4918A-BD56-4A8B-A59F-772BD6734ED0},9618.098623,4.882456e+06,POINT (548718.484 5286031.037)
4,5.0,0.0,57.157023,{40947E9A-D688-4C0D-A90B-70BCAA660DE0},7807.650622,2.489750e+06,POINT (549379.446 5286544.365)
...,...,...,...,...,...,...,...
3695,3696.0,0.0,985.847070,{2363DDFD-B98C-4A69-A871-C38D6885885C},30514.017509,4.294333e+07,POINT (532401.006 5275608.358)
3696,3697.0,0.0,2609.642770,{564442F1-C923-4F7D-8D18-44DDE3C9978F},43581.705129,1.136756e+08,POINT (532469.443 5272860.464)
3697,3698.0,0.0,641.622735,{A668A34C-6202-41A1-B839-736188CE6670},31374.018528,2.794898e+07,POINT (534285.03 5272495.224)
3698,3699.0,0.0,3095.122859,{273D851F-C04B-4029-8469-73AD6F8497C4},65860.215701,1.348230e+08,POINT (537281.343 5273180.28)


In [11]:
#Making the spatial join
ZIP_code = ZIP_code.to_crs(epsg = epsg_work)
TAZ_to_ZIP = gpd.sjoin(TAZ, ZIP_code, how='left', predicate='intersects')

#Counting if there is any NA
print(f'We have {TAZ_to_ZIP['GEOID10'].isna().sum()} TAZs that dont have a ZIP code')

#Creating a dataframe that has all the TAZs that are not assigned to anything
Missing_TAZ = TAZ_to_ZIP[TAZ_to_ZIP['GEOID10'].isna()]

We have 94 TAZs that dont have a ZIP code


In [14]:
#Getting the whole geometry and not just the centroids
Missing_TAZ_a = TAZ_original.loc[
    TAZ_original['taz'].isin(Missing_TAZ['taz'])].to_crs(epsg_work)

#Making an intersect because I'm lazy
TAZ_to_ZIP_missing = gpd.sjoin(Missing_TAZ_a, ZIP_code, how='left', predicate='intersects')

#Assigning it to the first one that intersects it because im lazy
TAZ_to_ZIP_missing = TAZ_to_ZIP_missing.groupby('taz').first().reset_index()

#Creating a dataframe that has all the assigned TAZ's and zipcodes with the centroid method
#And getting the original geometry
TAZ_to_ZIP_NONA = TAZ_to_ZIP.dropna()
TAZ_to_ZIP_NONA.loc[:,'geometry'] = TAZ_original[TAZ_original['taz'].isin(TAZ_to_ZIP_NONA['taz'])].loc[:,'geometry']
TAZ_to_ZIP_NONA

,taz,park,acres,GlobalID,Shape_Length,Shape_Area,geometry,index_right,GEOID10
1,2.0,0.0,120.755587,{D5383690-0E2A-43B7-81DC-A17CFA9645B0},10591.087634,5.260092e+06,"MULTIPOLYGON (((1265788.093 268180.249, 126577...",20801.0,98133
2,3.0,0.0,124.332936,{4A65D6B2-0DFD-447F-B74C-8A2233E81DBF},9555.337706,5.415921e+06,"MULTIPOLYGON (((1265848.689 271492.146, 126617...",20801.0,98133
3,4.0,0.0,112.086215,{21D4918A-BD56-4A8B-A59F-772BD6734ED0},9618.098623,4.882456e+06,"MULTIPOLYGON (((1267095.215 269151.901, 126728...",20801.0,98133
4,5.0,0.0,57.157023,{40947E9A-D688-4C0D-A90B-70BCAA660DE0},7807.650622,2.489750e+06,"MULTIPOLYGON (((1270732.726 269633.63, 1271063...",20801.0,98133
5,6.0,0.0,64.596333,{9F6F447D-0DEF-43CB-AC31-C7B30B215781},8039.365290,2.813805e+06,"MULTIPOLYGON (((1271071.927 270062.991, 127106...",20801.0,98133
...,...,...,...,...,...,...,...,...,...
3694,3695.0,0.0,881.314909,{90009063-1302-4196-89A7-396E9A7C87E8},29909.701955,3.838992e+07,"MULTIPOLYGON (((1220136.14 239284.827, 1220912...",20787.0,98110
3695,3696.0,0.0,985.847070,{2363DDFD-B98C-4A69-A871-C38D6885885C},30514.017509,4.294333e+07,"MULTIPOLYGON (((1213639.678 239372.617, 121371...",20787.0,98110
3696,3697.0,0.0,2609.642770,{564442F1-C923-4F7D-8D18-44DDE3C9978F},43581.705129,1.136756e+08,"MULTIPOLYGON (((1212031.956 234139.577, 121203...",20787.0,98110
3697,3698.0,0.0,641.622735,{A668A34C-6202-41A1-B839-736188CE6670},31374.018528,2.794898e+07,"MULTIPOLYGON (((1219221.496 231349.083, 121923...",20787.0,98110


In [15]:
#Meshing both methods to get all TAZs assigned to their ZIPCodes
TAZ_to_ZIP = pd.concat([TAZ_to_ZIP_NONA, TAZ_to_ZIP_missing])
# TAZ_to_ZIP.isna().sum()
TAZ_to_ZIP.shape[0]
#WE GOT EM BOSS

C:\Users\olive\anaconda3\envs\OD_matrix2\Lib\site-packages\geopandas\array.py:1754: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / UTM zone 10N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


3700

In [16]:
#List of all the ZIP codes used
Unique_ZIPS = TAZ_to_ZIP['GEOID10'].unique()
Unique_ZIPS = sorted(Unique_ZIPS.astype(int).tolist())
Unique_ZIPS

[98001,
 98002,
 98003,
 98004,
 98005,
 98006,
 98007,
 98008,
 98010,
 98011,
 98012,
 98014,
 98019,
 98020,
 98021,
 98022,
 98023,
 98024,
 98026,
 98027,
 98028,
 98029,
 98030,
 98031,
 98032,
 98033,
 98034,
 98036,
 98037,
 98038,
 98039,
 98040,
 98042,
 98043,
 98045,
 98047,
 98050,
 98051,
 98052,
 98053,
 98055,
 98056,
 98057,
 98058,
 98059,
 98065,
 98068,
 98070,
 98072,
 98074,
 98075,
 98077,
 98087,
 98092,
 98101,
 98102,
 98103,
 98104,
 98105,
 98106,
 98107,
 98108,
 98109,
 98110,
 98112,
 98115,
 98116,
 98117,
 98118,
 98119,
 98121,
 98122,
 98125,
 98126,
 98133,
 98134,
 98136,
 98144,
 98146,
 98148,
 98155,
 98158,
 98166,
 98168,
 98177,
 98178,
 98188,
 98195,
 98198,
 98199,
 98201,
 98203,
 98204,
 98205,
 98207,
 98208,
 98223,
 98241,
 98251,
 98252,
 98258,
 98270,
 98271,
 98272,
 98275,
 98282,
 98288,
 98290,
 98292,
 98294,
 98296,
 98303,
 98304,
 98310,
 98311,
 98312,
 98314,
 98315,
 98321,
 98323,
 98327,
 98328,
 98329,
 98332,
 98333,


In [17]:
#Creating a dictionary where the keys are the Zipcodes and the values are the vectors that have the TAZs assigned to that zipcode
ZIP_dict = {}
for ZIP in Unique_ZIPS:
    ZIP_dict[f'{ZIP}'] = TAZ_to_ZIP.loc[TAZ_to_ZIP['GEOID10'] == str(ZIP), 'taz'].astype(int).tolist()


# Assigning the OD to each zipcode

In [18]:
#Creating the OD matrix
OD_matrix = pd.DataFrame()

In [19]:
#Loading the OD thing
# OD_PSRC = h5py.File('data/PSRC/17to18.h5', 'r')
# OD_SOV = pd.DataFrame(OD_PSRC['sov_inc2'][:,:])

#I believe the examples they gave us are wrong, gonna use the _trip.tsv file they provided to get the OD matrix at the TAZ level
#Done in OD_matrix_TAZ file

### Trying the aggregation method I devised

In [20]:
# random_data = np.random.rand(12, 12).round(decimals=1)*10
#
# df_rand = pd.DataFrame(random_data)
# df_rand

In [21]:
# df_rand_dict ={
#     '100': [0, 1],
#     '200': [2, 3,4,5],
#     '300': [6, 7,8,9],
#     '400': [10, 11],
#     }

In [22]:
# agg_df_rand = pd.DataFrame()
# for keys_row, values_row in df_rand_dict.items():
#     for keys_col, values_col in df_rand_dict.items():
#         agg_df_rand.loc[keys_row, keys_col] = df_rand.loc[
#             df_rand_dict[keys_row],
#             df_rand_dict[keys_col]
#         ].sum().sum()
#
# agg_df_rand

## Actually Aggregating

Gonna use the **OD_8to9** that I created from the *OD_matrix_TAZ* file in this project

In [23]:
OD_SOV = pd.read_csv('data/PSRC/PSRC_OO/8to9_SOV.csv')
OD_SOV.index = pd.RangeIndex(1,3837,step=1)
OD_SOV

,1,2,3,4,5,6,7,8,9,10,...,3896,3898,3899,3900,3902,3903,3904,3905,3907,3908
1,0,0,1,3,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,4,4,6,5,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,6,6,7,7,1,6,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,6,10,16,5,1,12,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1,2,5,2,3,0,7,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
for zip1, rows in ZIP_dict.items():
    for zip2, cols in ZIP_dict.items():
        OD_matrix.loc[zip1, zip2] = OD_SOV.iloc[
            ZIP_dict[zip1],
            ZIP_dict[zip2]
        ].sum().sum()

C:\Users\olive\AppData\Local\Temp\ipykernel_4088\2549654972.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  OD_matrix.loc[zip1, zip2] = OD_SOV.iloc[
C:\Users\olive\AppData\Local\Temp\ipykernel_4088\2549654972.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  OD_matrix.loc[zip1, zip2] = OD_SOV.iloc[
C:\Users\olive\AppData\Local\Temp\ipykernel_4088\2549654972.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

In [26]:
OD_matrix.to_csv('OD_8to9_SOV.csv')

PermissionError: [Errno 13] Permission denied: 'OD_8to9_SOV.csv'

In [61]:
ZIP_code_original = ZIP_code_original.to_crs(epsg = 4326)
Unique_ZIPS_str = [str(zip_code) for zip_code in Unique_ZIPS]

In [62]:
ZIP_code_original['GEOID10'] = ZIP_code_original['GEOID10'].astype(str)
ZIP_SEA = ZIP_code_original[ZIP_code_original['GEOID10'].isin(Unique_ZIPS_str)]
ZIP_SEA

,GEOID10,geometry
268,98408,"MULTIPOLYGON (((-122.46379 47.18469, -122.4622..."
272,98007,"MULTIPOLYGON (((-122.15494 47.58463, -122.1540..."
305,98353,"MULTIPOLYGON (((-122.50492 47.54653, -122.4914..."
582,98065,"MULTIPOLYGON (((-121.90872 47.52952, -121.908 ..."
686,98020,"MULTIPOLYGON (((-122.39681 47.78238, -122.3921..."
...,...,...
32176,98087,"MULTIPOLYGON (((-122.25621 47.87844, -122.2523..."
32537,98101,"MULTIPOLYGON (((-122.34494 47.60912, -122.3343..."
32626,98223,"MULTIPOLYGON (((-122.27627 48.15199, -122.2753..."
32930,98022,"MULTIPOLYGON (((-122.11324 47.21778, -122.1138..."


In [66]:
m = folium.Map(location= [47.605744, -122.334862])
folium.GeoJson(
    ZIP_SEA,
    popup= folium.GeoJsonPopup(
        fields = ['GEOID10']
    )
).add_to(m)
m